In [188]:
import torch
from PIL import Image
from torchvision import transforms
import cv2
import numpy as np

In [189]:
# Загрузка модели
ckpt = torch.load("fire_l.pt")
torch.save(ckpt, "updated-fire_l.pt")
model_path = 'updated-fire_l.pt'
checkpoint = torch.load(model_path)
model = checkpoint['model']

In [190]:
# Загрузка изображения
# image_path = "forest_fire/Testing/fire/abc186.jpg"
image_path = "photo_440337.jpeg"

image = cv2.imread(image_path)

# Проверка, что изображение загружено успешно
if image is None:
    print("Ошибка загрузки изображения.")
    exit()

In [191]:
# Сжимаем изображение до размера, ожидаемого моделью (416x416)
resized_image = cv2.resize(image, (416, 416))

In [192]:
# Преобразование изображения в тензор
transform = transforms.Compose([transforms.ToTensor()])
image_tensor = transform(resized_image).unsqueeze(0)  # добавляем размерность пакета

In [193]:
# Проверка, если входные данные имеют тип torch.FloatTensor или torch.HalfTensor
if image_tensor.dtype == torch.float32:
    image_tensor = image_tensor.to(torch.float32)
elif image_tensor.dtype == torch.float16:
    image_tensor = image_tensor.to(torch.float16)
else:
    print("Ошибка: Неподдерживаемый тип данных входных данных.")


In [194]:
# Приведение типа весов модели к типу данных входных данных
model.float()

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
    

In [195]:
# Получение предсказаний от модели
with torch.no_grad():
    predictions = model(image_tensor)
    # predictions = model(image)

In [196]:
print(image_tensor.shape)
print('\n')
print(predictions[0][0][5])
print('\n')
print(predictions[1][2][0])

torch.Size([1, 3, 416, 416])


tensor([2.0450e-06, 1.2801e-06, 6.8782e-07,  ..., 1.1849e-06, 1.1399e-06, 1.5464e-06])


tensor([[[  5.1848,   3.0602,   1.7905,  ...,  -1.4012,  -1.1076,  -0.0524],
         [  7.4375,   3.7793,   2.0810,  ...,  -0.5612,  -0.2043,   0.7088],
         [  8.3105,   3.9923,   2.1488,  ...,  -0.6246,  -0.3438,   0.6688],
         ...,
         [  5.2987,   3.1845,   2.2864,  ...,   1.0728,   1.7800,   2.8558],
         [  4.9206,   3.0306,   2.2343,  ...,   1.3627,   1.8848,   2.6517],
         [  3.9895,   2.6277,   2.0065,  ...,   1.4837,   1.8370,   2.2928]],

        [[  4.9789,   5.9213,   3.1802,  ...,  -0.2564,  -0.4969,   0.8603],
         [  7.3107,   7.3591,   3.7389,  ...,   0.4791,   0.0847,   1.7617],
         [  8.2373,   8.0441,   4.0341,  ...,   0.6507,   0.2713,   2.1608],
         ...,
         [  5.0739,   5.1609,   2.8645,  ...,   1.0810,   1.6414,   3.0354],
         [  4.6737,   4.6512,   2.6553,  ...,   1.2854,   1.7321,   2.7215],
   

In [197]:
def draw_boxes(image, predictions):
    # Копируем изображение, чтобы не изменять оригинал
    image_with_boxes = image.copy()

    # Переменные для проверки наличия огня или дыма
    fire_detected = False
    smoke_detected = False

    # Проход по всем обнаруженным объектам
    for pred in predictions[0][0]:
        # Извлечение координат прямоугольника и уверенности
        x_center, y_center, width, height = pred[:4]

        # Определение цвета прямоугольника в зависимости от типа объекта
        color = (255, 0, 0) if pred[5] == 0 else (192, 192, 192)  # Красный для огня, серый для дыма

        # Преобразование координат в координаты верхнего левого угла и размеры
        x1 = int((x_center - width / 2) * image.shape[1])
        y1 = int((y_center - height / 2) * image.shape[0])
        x2 = int((x_center + width / 2) * image.shape[1])
        y2 = int((y_center + height / 2) * image.shape[0])

        # Рисуем прямоугольник на изображении
        cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), color, 2)

        # Определение текста для подписи в зависимости от типа объекта
        label = "Огонь" if pred[5] == 0 else "Дым"

        # Добавляем подпись к прямоугольнику
        cv2.putText(image_with_boxes, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Проверяем, обнаружен ли огонь или дым
        if pred[5] == False:
            fire_detected = True
        else:
            smoke_detected = True

    # Показываем изображение с нарисованными прямоугольниками и подписями
    cv2.imshow("Image with Boxes", image_with_boxes)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Проверяем, был ли обнаружен огонь и/или дым и выводим соответствующее сообщение
    if fire_detected and smoke_detected:
        print("На изображении обнаружены огонь и дым.")
    if fire_detected:
        print("На изображении обнаружен огонь.")
    if smoke_detected:
        print("На изображении обнаружен дым.")
    else:
        print("На изображении не обнаружено огня и дыма.")

In [198]:
# def draw_boxes(image, predictions):
#     # Копируем изображение, чтобы не изменять оригинал
#     image_with_boxes = image.copy()

#     # Проходим по всем предсказаниям
#     for pred in predictions[0][0]:
#         # Извлекаем координаты и уверенность прямоугольника из предсказания
#         x_center, y_center, width, height, confidence = pred.tolist()

#         # Преобразуем координаты в координаты верхнего левого угла
#         x1 = int((x_center - width / 2) * image.shape[1])
#         y1 = int((y_center - height / 2) * image.shape[0])
#         x2 = int((x_center + width / 2) * image.shape[1])
#         y2 = int((y_center + height / 2) * image.shape[0])

#         # Рисуем прямоугольник на изображении
#         cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2)

#         # Добавляем текст с уверенностью
#         text = f"Confidence: {confidence:.2f}"
#         cv2.putText(image_with_boxes, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#     # Показываем изображение с нарисованными прямоугольниками
#     cv2.imshow("Image with Boxes", image_with_boxes)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


In [199]:
# Вызов функции для рисования прямоугольников на изображении с предсказаниями
draw_boxes(image, predictions)

На изображении обнаружен дым.
